# Getting data from APIs

Você não pode construir um modelo sem dados, certo? Em projetos anteriores, trabalhamos com dados armazenados em arquivos (como um CSV) ou bancos de dados (como SQL). Neste projeto, vamos obter nossos dados de um servidor web usando uma API.

Então, nesta lição, vamos aprender o que é uma API e como extrair dados de uma. Também vamos trabalhar na transformação de nossos dados em um formato gerenciável. Vamos lá!

In [1]:
import pandas as pd
import requests

# Acessando APIs através de uma URL

Nesta lição, vamos extrair informações do mercado de ações da API [AlphaVantage](https://alphavantage.co/). Para entender como uma API funciona, considere a URL abaixo.

Tire um momento para ler o texto do link em si, depois clique nele e examine os dados que aparecem no seu navegador. Qual é o formato dos dados? Quais dados estão incluídos? Como eles estão organizados?

https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo

Observe que esta URL possui vários componentes. Vamos analisá-los um por um.

| URL | Componente |
|:--- | :-------- |
| `https://www.alphavantage.co` | Este é o **hostname** ou **URL base**. É o endereço web do servidor onde podemos obter nossos dados de ações. |
| `/query` | Este é o **caminho**. A maioria das APIs tem várias operações diferentes que podem ser feitas. O caminho é o nome da operação específica que queremos acessar. |
| `?` | Este ponto de interrogação indica que tudo o que segue na URL é um **parâmetro**. Cada parâmetro é separado por um caractere `&`. Esses parâmetros fornecem informações adicionais que irão modificar o comportamento da operação. Isso é semelhante à forma como passamos **argumentos** para funções em Python. |
| `function=TIME_SERIES_DAILY` | Nosso primeiro parâmetro usa a palavra-chave `function`. O valor é `TIME_SERIES_DAILY`. Neste caso, estamos pedindo dados de ações **diários**. |
| `symbol=IBM` | Nosso segundo parâmetro usa a palavra-chave `symbol`. Portanto, estamos pedindo dados de uma ação cujo [**ticker symbol**](https://en.wikipedia.org/wiki/Ticker_symbol) é `IBM`. |
| `apikey=demo` | Da mesma forma que você precisa de uma senha para acessar alguns sites, uma **chave de API** ou **token de API** é a senha que você usará para acessar a API. |

Agora que temos uma noção dos componentes da URL que obtém informações do AlphaVantage, vamos criar a nossa própria para uma ação diferente.

### Exercício:
Usando a URL acima como modelo, crie uma nova URL para obter os dados da [Ambuja Cement](https://www.ambujacement.com/). com o ticker sendo `"AMBUJACEM.BSE"`.


In [2]:
url = (
    "https://www.alphavantage.co"    # URL base
    "/query?"                        # caminho
    "function=TIME_SERIES_DAILY&"    # param
    "symbol=AMBUJACEM.BSE&"          # param
    "apikey=demo"                    # param
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&apikey=demo'

Oh não! Um problema. Parece que precisamos da nossa própria chave de API para acessar os dados.

Como você pode imaginar, uma chave de API é uma informação que deve ser mantida em sigilo, então é uma má ideia incluí-la no código da nossa aplicação. Quando se trata de informações sensíveis como esta, desenvolvedores e cientistas de dados as armazenam como uma [variável de ambiente](https://en.wikipedia.org/wiki/Environment_variable) que é mantida em um arquivo `.env`.

### Exercício:
Obtenha sua chave de API e a salve em seu arquivo `.env`.

Agora que armazenamos nossa chave de API, precisamos importá-la para o nosso código. Isso é comumente feito criando um módulo `config`.

### Exercício:
Importe a variável `settings` do módulo `config`. Em seguida, use o comando `dir` para ver quais atributos ela possui.

In [10]:
# Import settings
# !pip install pydantic
from config import settings

# Use `dir` to list attributes
dir(settings)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_validator__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',

In [ ]:
settings.alpha_api_key

### Exercício:
Crie uma nova URL para `"AMBUJACEM.BSE"`:


In [11]:
url = (
    "https://www.alphavantage.co"
    "/query?"
    "function=TIME_SERIES_DAILY&"
    "symbol=AMBUJACEM.BSE&"
    f"apikey={settings.alpha_api_key}"
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&apikey=JWK8Z958ODBXS114'

Está funcionando! Acontece que há muitos mais parâmetros. Vamos construir nossa URL para incluí-los.

### Exercício:
Vá até a documentação da [API AlphaVantage Time Series Daily](https://www.alphavantage.co/documentation/#daily). Amplie sua URL para incorporar todos os parâmetros listados na documentação. Além disso, para tornar sua URL mais dinâmica, crie nomes de variáveis para todos os parâmetros que podem ser adicionados à URL.

In [17]:
ticker = "AMBUJACEM.BSE"
output_size = "compact"
data_type = "json"

url = (
    "https://www.alphavantage.co"
    "/query?"
    "function=TIME_SERIES_DAILY&"
    f"symbol={ticker}&"
    f"outputsize={output_size}&"
    f"datatype={data_type}&"
    f"apikey={settings.alpha_api_key}"
)

print("url type:", type(url))
url

url type: <class 'str'>


'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=AMBUJACEM.BSE&outputsize=compact&datatype=json&apikey=JWK8Z958ODBXS114'

# Acessando APIs Através de uma Solicitação

Vimos como acessar a API AlphaVantage clicando em uma URL, mas isso não funcionará para a aplicação que estamos construindo neste projeto, pois apenas humanos clicam em URLs. Programas de computador acessam APIs fazendo **solicitações**. Vamos construir nossa primeira solicitação usando a URL que criamos na tarefa anterior.

### Exercício:
Use a biblioteca `requests` para fazer uma solicitação `get` à URL que você criou na tarefa anterior. Atribua a resposta à variável `response`.

In [18]:
response = requests.get(url=url)

print("response type:", type(response))

response type: <class 'requests.models.Response'>


Isso nos informa que tipo de resposta recebemos, mas não nos diz nada sobre o que isso significa. Se quisermos descobrir quais tipos de dados estão realmente *na* resposta, precisaremos usar o comando `dir`.

### Exercício:
Use o comando `dir` para ver quais atributos e métodos a variável `response` possui.

In [20]:
# Use `dir` on your `response`
dir(response)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

O comando `dir` retorna uma lista e, como você pode ver, há muitas possibilidades aqui! Por enquanto, vamos nos concentrar em dois atributos: `status_code` e `text`.

Começaremos com `status_code`. Sempre que você faz uma chamada para uma URL, a resposta inclui um [código de status HTTP](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes) que pode ser acessado com o atributo `status_code`. Vamos ver qual é o nosso.

### Exercício:
Atribua o código de status da sua `response` à variável `response_code`.

In [34]:
response_code = response.status_code

print("code type:", type(response_code))
print(response_code)

code type: <class 'int'>
200


Traduzido para o inglês, `200` significa "OK". É a resposta padrão para uma solicitação HTTP bem-sucedida. Em outras palavras, funcionou! Recebemos com sucesso os dados da API AlphaVantage.

Agora vamos dar uma olhada no `text`.

### Exercício:
Atribua o texto da sua `response` à variável `response_text`.

In [35]:
response_text = response.text

print("response_text type:", type(response_text))
print(response_text[:200])

response_text type: <class 'str'>
{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "AMBUJACEM.BSE",
        "3. Last Refreshed": "2024-10-15",
        "4. Output 


Essa string parece com os dados que vimos anteriormente em nosso navegador quando clicamos na URL no exercício 5. Mas não podemos trabalhar com dados estruturados como JSON quando estão como uma string. Em vez disso, precisamos que eles estejam em um dicionário.

### Exercício:
Use o método `json` para acessar uma versão em dicionário dos dados. Atribua-a à variável chamada `response_data`.

In [36]:
response_data = response.json()

print("response_data type:", type(response_data))

response_data type: <class 'dict'>


Vamos verificar se os dados estão estruturados da mesma forma que vimos em nosso navegador.

### Exercício:
Imprima as chaves de `response_data`. Elas são o que você esperava?

In [37]:
# Print `response_data` keys
response_data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

Agora vamos olhar os dados que estão atribuídos à chave `"Time Series (Daily)"`.

### Exercício:
Atribua o valor da chave `"Time Series (Daily)"` à variável `stock_data`. Em seguida, examine os dados de um dos dias em `stock_data`.

In [ ]:
# Extract `"Time Series (Daily)"` value from `response_data`
stock_data = ...

print("stock_data type:", type(stock_data))

# Extract data for one of the days in `stock_data`



Agora que sabemos como os dados estão organizados quando os extraímos da API, vamos transformá-los em um DataFrame para torná-los mais gerenciáveis.

### Exercício:
Leia os dados de `stock_data` em um DataFrame chamado `df_ambuja`. Certifique-se de que todos os seus tipos de dados estão corretos!

In [ ]:
df_ambuja = ...

print("df_ambuja shape:", df_ambuja.shape)
print()
print(df_ambuja.info())
df_ambuja.head(10)

Você notou que o índice para `df_ambuja` não tem uma entrada para todos os dias? Considerando que se trata de dados do mercado de ações, por que você acha que isso acontece?

No geral, isso parece muito bom, mas há alguns problemas: o tipo de dados das datas e o formato dos cabeçalhos. Vamos corrigir as datas primeiro. No momento, as datas são strings; para que o restante do nosso código funcione, precisaremos criar um `DatetimeIndex` adequado.

### Exercício:
Transforme o índice de `df_ambuja` em um `DatetimeIndex` com o nome `"date"`.

In [ ]:
# Convert `df_ambuja` index to `DatetimeIndex`


# Name index "date"


print(df_ambuja.info())
df_ambuja.head()

__Nota:__ as linhas em `df_ambuja` estão ordenadas de forma <b>decrescente</b>, com a data mais recente no topo. Isso será vantajoso quando armazenarmos e recuperarmos os dados do nosso banco de dados da aplicação, mas precisaremos ordená-los de forma <b>crescente</b> antes de podermos usá-los para treinar um modelo.

Ok! Agora que as datas estão corrigidas, vamos lidar com os cabeçalhos. Não há nada realmente *errado* com eles, mas aqueles números os fazem parecer um pouco inacabados. Vamos nos livrar deles.

### Exercício:
Remova a numeração dos nomes das colunas de `df_ambuja`.

In [ ]:
# Remove numbering from `df_ambuja` column names
df_ambuja.columns = ...

print(df_ambuja.info())
df_ambuja.head()

# Programação Defensiva

Programação defensiva é a prática de escrever código que continuará a funcionar, mesmo que algo dê errado. Nunca conseguiremos prever todos os problemas que as pessoas podem encontrar com nosso código, mas podemos tomar medidas para garantir que as coisas não desmoronem sempre que um desses problemas acontecer.

Até agora, fizemos solicitações de API onde tudo funciona. Mas erros de codificação e problemas com servidores são comuns, e podem causar grandes problemas em um projeto de ciência de dados. Vamos ver como nossa `response` muda quando introduzimos erros comuns em nosso código.

### Exercício:
Volte ao exercício 5 e mude a primeira parte da sua URL. Em vez de `"query"`, use `"search"` (um caminho que não existe). Em seguida, execute novamente seu código para todas as tarefas que se seguem. O que muda? O que permanece o mesmo?

Sabemos o que acontece quando tentamos acessar um endereço inválido. Mas e quando acessamos o *caminho correto* com um símbolo de ação *inválido*?

### Exercício:
Volte ao exercício 5 e mude o símbolo da ação de `"AMBUJACEM.BSE"` para `"RAMBUJACEM.BSE"` (uma empresa que não existe). Em seguida, execute novamente seu código para todas as tarefas que se seguem. Novamente, observe o que muda e o que permanece o mesmo.

Vamos formalizar nosso processo de extração e transformação para a API AlphaVantage em uma função reproduzível.

### Exercício:
Construa uma função `get_daily` que obtenha dados da API AlphaVantage e retorne um DataFrame limpo. Use a docstring como guia. Quando estiver satisfeito com o resultado, envie seu trabalho para o avaliador.

In [ ]:
def get_daily():

    """Get daily time series of an equity from AlphaVantage API.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the equity.
    output_size : str, optional
        Number of observations to retrieve. "compact" returns the
        latest 100 observations. "full" returns all observations for
        equity. By default "full".

    Returns
    -------
    pd.DataFrame
        Columns are 'open', 'high', 'low', 'close', and 'volume'.
        All are numeric.
    """
    # Create URL (8.1.5)


    # Send request to API (8.1.6)


    # Extract JSON data from response (8.1.10)


    # Read data into DataFrame (8.1.12 & 8.1.13)


    # Convert index to `DatetimeIndex` named "date" (8.1.14)


    # Remove numbering from columns (8.1.15)


    # Return DataFrame
    return df

In [ ]:
# Test your function
df_ambuja = get_daily(ticker="AMBUJACEM.BSE")

print(df_ambuja.info())
df_ambuja.head()

Como essa função lida com os dois erros que exploramos nesta seção? Nosso primeiro erro, uma URL inválida, é algo com o qual não precisamos nos preocupar. Não importa o que o usuário insira nesta função, a URL sempre estará correta. Mas veja o que acontece quando o usuário insere um símbolo de ação inválido. Qual é a mensagem de erro? Isso ajudaria o usuário a localizar seu erro?

### Exercício:
Adicione uma cláusula `if` à sua função `get_daily` para que ela lance uma `Exception` quando um usuário fornecer um símbolo de ação inválido. Certifique-se de que a mensagem de erro seja informativa.

In [ ]:
# Test your Exception
df_test = get_daily(ticker="ABUJACEM.BSE")

Certo! Agora temos todas as ferramentas necessárias para obter os dados para o nosso projeto. Na próxima lição, tornaremos nosso código AlphaVantage mais reutilizável, criando um módulo `data` com definições de classe. Também criaremos o código necessário para armazenar e ler esses dados do nosso banco de dados de aplicação.

# Desenvolvimento Orientado a Testes (TDD)

Na lição anterior, aprendemos como obter dados de uma API. Nesta lição, temos dois objetivos. Primeiro, vamos pegar o código que usamos para acessar a API e construir uma classe `AlphaVantageAPI`. Isso nos permitirá reutilizar nosso código. Segundo, criaremos uma classe `SQLRepository` que nos ajudará a carregar nossos dados de ações em um banco de dados SQLite e depois extraí-los para uso posterior. Além disso, construiremos esse código usando uma técnica chamada **desenvolvimento orientado a testes**, onde usaremos instruções `assert` para garantir que tudo esteja funcionando corretamente. Assim, evitaremos problemas mais tarde ao construir nossa aplicação.

In [ ]:
%load_ext autoreload
%load_ext sql
%autoreload 2

import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
from config import settings

# Construindo Nosso Módulo de Dados

Para nossa aplicação, vamos manter todas as classes que usamos para extrair, transformar e carregar dados em um único módulo que chamaremos de `data`.

## Classe AlphaVantage API

Vamos começar pegando o código que criamos na última lição e incorporá-lo em uma classe que será responsável por obter dados da API AlphaVantage.

### Exercício:
No módulo `data`, crie uma definição de classe para `AlphaVantageAPI`. Por enquanto, certifique-se de que ela tenha um método `__init__` que anexe sua chave de API como o atributo `__api_key`. Depois de terminar, importe a classe abaixo e crie uma instância dela chamada `av`.

In [ ]:
# Import `AlphaVantageAPI`


# Create instance of `AlphaVantageAPI` class
av = ...

print("av type:", type(av))

Lembre-se da função `get_daily` que fizemos na última lição? Agora vamos transformá-la em um método de classe.

### Exercício:
Crie um método `get_daily` para sua classe `AlphaVantageAPI`. Assim que terminar, use a célula abaixo para buscar os dados das ações da empresa de energia renovável [Suzlon](https://www.suzlon.com/) e atribua-os ao DataFrame `df_suzlon`.

In [ ]:
# Define Suzlon ticker symbol
ticker = "SUZLON.BSE"

# Use your `av` object to get daily data
df_suzlon = ...

print("df_suzlon type:", type(df_suzlon))
print("df_suzlon shape:", df_suzlon.shape)
df_suzlon.head()

Certo! A próxima coisa que precisamos fazer é testar nosso novo método para garantir que ele funcione da maneira que desejamos. Normalmente, esses tipos de testes são escritos *antes* de escrever o método, mas, neste primeiro caso, faremos o contrário para entender melhor como as instruções de assert funcionam.

### Exercício:
Crie quatro instruções `assert` para testar a saída do seu método `get_daily`. Use os comentários abaixo como guia.

In [ ]:
# Does `get_daily` return a DataFrame?


# Does DataFrame have 5 columns?


# Does DataFrame have a DatetimeIndex?


# Is the index name "date"?


### Exercício:
Crie mais dois testes para a saída do seu método `get_daily`. Use os comentários abaixo como guia.

In [ ]:
# Does DataFrame have correct column names?


# Are columns correct data type?


Ok! Agora que nossa classe `AlphaVantageAPI` está pronta para obter dados, vamos focar na classe que precisaremos para armazenar nossos dados em nosso banco de dados SQLite.

## Classe Repositório SQL

Não seria eficiente se nosso aplicativo precisasse obter dados da API AlphaVantage toda vez que quiséssemos explorar nossos dados ou construir um modelo, então precisaremos armazenar nossos dados em um banco de dados. Como nossos dados são altamente estruturados (cada DataFrame que extraímos da AlphaVantage sempre terá as mesmas cinco colunas), faz sentido usar um banco de dados SQL.

Usaremos o SQLite para nosso banco de dados. Para consistência, esse banco de dados sempre terá o mesmo nome, que armazenamos em nosso arquivo `.env`.

### Exercício:
Conecte-se ao banco de dados cujo nome está armazenado no arquivo `.env` deste projeto. Certifique-se de definir o argumento `check_same_thread` como `False`. Atribua a conexão à variável `connection`.

In [ ]:
connection = ...

print("connection type:", type(connection))

Temos uma conexão, e agora precisamos começar a construir a classe que irá gerenciar todas as nossas transações com o banco de dados. Com esta classe, no entanto, vamos criar nossos testes *antes* de escrever a definição da classe.

### Exercício:
Escreva dois testes para a classe `SQLRepository`, usando os comentários abaixo como guia.

In [ ]:
# Import class definition


# Create instance of class
repo = ...

# Does `repo` have a "connection" attribute?


# Is the "connection" attribute a SQLite `Connection`?


__Dica:__ Você não poderá executar este bloco de código ☝️ até completar a tarefa abaixo. 👇

### Exercício:
Crie uma definição para sua classe `SQLRepository`. Por enquanto, complete apenas o método `__init__`. Assim que terminar, use o código que você escreveu na tarefa anterior para testá-la.

O próximo método que precisamos para a classe `SQLRepository` é um que nos permita armazenar informações. No jargão SQL, isso é geralmente referido como **inserir** tabelas no banco de dados.

### Exercício:
Adicione um método `insert_table` à sua classe `SQLRepository`. Como guia, use as instruções `assert` abaixo e o docstring no módulo `data`. Quando terminar, execute a célula abaixo para verificar seu trabalho.

In [ ]:
response = repo.insert_table(table_name=ticker, records=df_suzlon, if_exists="replace")

# Does your method return a dictionary?
assert isinstance(response, dict)

# Are the keys of that dictionary correct?
assert sorted(list(response.keys())) == ["records_inserted", "transaction_successful"]

Se nosso método estiver passando as instruções `assert`, sabemos que ele está retornando um registro da transação no banco de dados, mas ainda precisamos verificar se os dados foram realmente adicionados ao banco de dados.

### Exercício:
Escreva uma consulta SQL para obter as **cinco primeiras linhas** da tabela de dados da Suzlon que você acabou de inserir no banco de dados.

In [ ]:

%sql sqlite:///

In [ ]:
%%sql



Podemos inserir dados no nosso banco de dados, mas não devemos esquecer que também precisamos ler dados dele. A leitura será um pouco mais complexa do que a inserção, então vamos começar escrevendo o código neste notebook antes de incorporá-lo à nossa classe `SQLRepository`.

### Exercício:
Primeiro, escreva uma consulta SQL para obter **todos** os dados da Suzlon. Em seguida, use o pandas para extrair os dados do banco de dados e ler em um DataFrame, chamado `df_suzlon_test`.

In [ ]:
sql = ...
df_suzlon_test = ...

print("df_suzlon_test type:", type(df_suzlon_test))
print()
print(df_suzlon_test.info())
df_suzlon_test.head()

Agora que sabemos como ler uma tabela do nosso banco de dados, vamos transformar nosso código em uma função adequada. Mas, como estamos fazendo design reverso, precisamos começar com nossos testes.

### Exercício:
Complete as declarações `assert` abaixo para testar sua função `read_table`. Use os comentários como guia.

In [ ]:
# Assign `read_table` output to `df_suzlon`
df_suzlon = read_table(table_name="SUZLON.BSE", limit=2500)  # noQA F821

# Is `df_suzlon` a DataFrame?


# Does it have a `DatetimeIndex`?


# Is the index named "date"?


# Does it have 2,500 rows and 5 columns?


# Are the column names correct?


# Are the column data types correct?


# Print `df_suzlon` info
print("df_suzlon shape:", df_suzlon.shape)
print()
print(df_suzlon.info())
df_suzlon.head()

__Dica:__ Você não poderá executar este bloco de código ☝️ até completar a tarefa abaixo. 👇

### Exercício:
Expanda o código que você escreveu acima para completar a função `read_table` abaixo. Use a docstring como um guia.

__Dica:__ Lembre-se de que armazenamos nossos dados ordenados de forma <b>decrescente</b> pela data. Isso definitivamente facilitará a implementação do <code>read_table</code>!

In [ ]:
def read_table():

    """Read table from database.

    Parameters
    ----------
    table_name : str
        Name of table in SQLite database.
    limit : int, None, optional
        Number of most recent records to retrieve. If `None`, all
        records are retrieved. By default, `None`.

    Returns
    -------
    pd.DataFrame
        Index is DatetimeIndex "date". Columns are 'open', 'high',
        'low', 'close', and 'volume'. All columns are numeric.
    """
    # Create SQL query (with optional limit)
    sql = ...


    # Retrieve data, read into DataFrame
    df = ...

    # Return DataFrame
    return df

### Exercício:
Transforme a função `read_table` em um método da sua classe `SQLRepository`.

### Exercício:
Volte à tarefa 11 e altere o código para que você teste o método da sua classe em vez da função do notebook.

Excelente! Temos tudo o que precisamos para obter dados do AlphaVantage, salvar esses dados em nosso banco de dados e acessá-los mais tarde. Agora é hora de fazer uma pequena análise exploratória para comparar as ações das duas empresas para as quais temos dados.

# Comparar Retorno de Ações

Já temos os dados da Suzlon Energy em nosso banco de dados, mas precisamos adicionar os dados da Ambuja Cement antes de podermos comparar as duas ações.

### Exercício:
Use as instâncias das classes `AlphaVantageAPI` e `SQLRepository` que você criou nesta lição (`av` e `repo`, respectivamente) para obter os dados de ações da Ambuja Cement e armazená-los no banco de dados.

In [ ]:
ticker = "AMBUJACEM.BSE"

# Get Ambuja data using `av`
ambuja_records = ...

# Insert `ambuja_records` database using `repo`
response = ...

response

Vamos dar uma olhada nos dados para garantir que estamos obtendo o que precisamos.

### Exercício:
Usando o método `read_table` que você adicionou ao seu `SQLRepository`, extraia as 2.500 linhas mais recentes de dados da Ambuja Cement do banco de dados e atribua o resultado a `df_ambuja`.

In [ ]:
ticker = "AMBUJACEM.BSE"
df_ambuja = ...

print("df_ambuja type:", type(df_ambuja))
print("df_ambuja shape:", df_ambuja.shape)
df_ambuja.head()

Já passamos bastante tempo observando esses dados, mas o que eles realmente representam? Acontece que o mercado de ações é muito parecido com qualquer outro mercado: as pessoas compram e vendem bens. Os preços desses bens podem subir ou descer dependendo de fatores como oferta e demanda. No caso de um mercado de ações, os bens sendo vendidos são ações (também chamadas de "equities" ou "securities"), que representam uma participação na propriedade de uma corporação.

Durante cada dia de negociação, o preço de uma ação muda, então, quando estamos avaliando se uma ação pode ser um bom investimento, observamos quatro tipos de números: abertura (open), máximo (high), mínimo (low), fechamento (close) e volume (volume). **Abertura (Open)** é exatamente o que parece: o preço de venda de uma ação quando o mercado abre para o dia. Da mesma forma, **fechamento (Close)** é o preço de venda de uma ação quando o mercado fecha no final do dia, e **máximo (High)** e **mínimo (Low)** são os preços máximo e mínimo de uma ação ao longo do dia. **Volume** é o número de ações de uma determinada empresa que foram compradas e vendidas naquele dia. De modo geral, uma empresa cujas ações tiveram um grande volume de negociação verá mais variação de preço ao longo do dia do que uma empresa cujas ações foram negociadas de forma mais leve.

Vamos visualizar como o preço da Ambuja Cement mudou na última década.

### Exercício:
Plote o preço de fechamento de `df_ambuja`. Certifique-se de rotular seus eixos e incluir uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot `df_ambuja` closing price


# Label axes



# Add legend


Vamos adicionar o preço de fechamento da Suzlon ao nosso gráfico para que possamos comparar os dois.

### Exercício:
Crie um gráfico que mostre os preços de fechamento de `df_suzlon` e `df_ambuja`. Novamente, rotule seus eixos e inclua uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot `df_suzlon` and `df_ambuja`



# Label axes



# Add legend


Olhando para este gráfico, podemos concluir que a Ambuja Cement é uma ação "melhor" do que a Suzlon Energy, pois seu preço é mais alto. Mas o preço é apenas um fator que um investidor deve considerar ao criar uma estratégia de investimento. O que é definitivamente verdade é que é difícil fazer uma comparação direta entre essas duas ações, pois há uma diferença de preço tão grande.

Uma maneira de os investidores compararem ações é observando seus **retornos**. Um retorno é a mudança de valor em um investimento, representada como uma porcentagem. Então, vamos analisar os retornos diários de nossas duas ações.

### Exercício:
Adicione uma coluna `"return"` ao `df_ambuja` que mostre a mudança percentual na coluna `"close"` de um dia para o outro.

__Dica:__ Nossos dois DataFrames estão ordenados de forma <b>decrescente</b> por data, mas você precisará garantir que estejam ordenados de forma <b>crescente</b> para calcular seus retornos.

In [ ]:
# Sort DataFrame ascending by date


# Create "return" column


print("df_ambuja shape:", df_ambuja.shape)
print(df_ambuja.info())
df_ambuja.head()

### Exercício:
Adicione uma coluna `"return"` ao `df_suzlon` que mostre a variação percentual na coluna `"close"` de um dia para o outro.

In [ ]:
# Sort DataFrame ascending by date


# Create "return" column


print("df_suzlon shape:", df_suzlon.shape)
print(df_suzlon.info())
df_suzlon.head()

Agora vamos plotar os retornos das nossas duas empresas e ver como as duas se comparam.

### Exercício:
Plote os retornos de `df_suzlon` e `df_ambuja`. Certifique-se de rotular seus eixos e usar uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
# Plot returns for `df_suzlon` and `df_ambuja`



# Label axes



# Add legend


Sucesso! Ao representar os retornos como uma porcentagem, conseguimos comparar duas ações que têm preços muito diferentes. Mas o que essa visualização está nos dizendo? Podemos ver que os retornos da Suzlon têm uma variação maior. Observamos grandes ganhos e grandes perdas. Em contraste, a variação da Ambuja é mais estreita, o que significa que o preço não flutua tanto.

Outro nome para essa flutuação diária nos retornos é chamado de [**volatilidade**](https://en.wikipedia.org/wiki/Volatility_(finance)), que é outro fator importante para os investidores. Portanto, na próxima lição, aprenderemos mais sobre volatilidade e, em seguida, construiremos um modelo de séries temporais para prever isso.

# Previsão de Volatilidade

Na última aula, aprendemos que uma característica das ações que é importante para os investidores é a **volatilidade**. Na verdade, é tão importante que existem vários modelos de séries temporais para prevê-la. Nesta aula, construiremos um desses modelos chamado **GARCH**. Também continuaremos trabalhando com declarações de assertiva para testar nosso código.

In [ ]:
import sqlite3

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arch import arch_model
from config import settings
from data import SQLRepository
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Preparar Dados

Como sempre, a primeira coisa que precisamos fazer é nos conectar à nossa fonte de dados.

## Importação

### Exercício:
Crie uma conexão com o seu banco de dados e, em seguida, instancie um SQLRepository chamado repo para interagir com esse banco de dados.

In [ ]:
connection = ...
repo = ...

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

Agora que estamos conectados a um banco de dados, vamos extrair o que precisamos.

### Exercício:
Extraia as 2.500 linhas mais recentes de dados da Ambuja Cement do seu banco de dados. Atribua os resultados à variável `df_ambuja`.

In [ ]:
df_ambuja = ...

print("df_ambuja type:", type(df_ambuja))
print("df_ambuja shape:", df_ambuja.shape)
df_ambuja.head()

Para treinar nosso modelo, os únicos dados que precisamos são os retornos diários para `"AMBUJACEM.BSE"`. Aprendemos como calcular os retornos na última aula, mas agora vamos formalizar esse processo com uma função de tratamento.

### Exercício:
Crie uma função `wrangle_data` cujo resultado sejam os retornos de uma ação armazenada no seu banco de dados. Use a docstring como guia e as declarações de assertiva no bloco de código a seguir para testar sua função.

In [ ]:
def wrangle_data():

    """Extract table data from database. Calculate returns.

    Parameters
    ----------
    ticker : str
        The ticker symbol of the stock (also table name in database).

    n_observations : int
        Number of observations to return.

    Returns
    -------
    pd.Series
        Name will be `"return"`. There will be no `NaN` values.
    """
    # Get table from database


    # Sort DataFrame ascending by date


    # Create "return" column


    # Return returns
    return ...

Quando você executar a célula abaixo para testar sua função, também criará uma Série `y_ambuja` que usaremos para treinar nosso modelo.

In [ ]:
y_ambuja = wrangle_data(ticker="AMBUJACEM.BSE", n_observations=2500)

# Is `y_ambuja` a Series?
assert isinstance(y_ambuja, pd.Series)

# Are there 2500 observations in the Series?
assert len(y_ambuja) == 2500

# Is `y_ambuja` name "return"?
assert y_ambuja.name == "return"

# Does `y_ambuja` have a DatetimeIndex?
assert isinstance(y_ambuja.index, pd.DatetimeIndex)

# Is index sorted ascending?
assert all(y_ambuja.index == y_ambuja.sort_index(ascending=True).index)

# Are there no `NaN` values?
assert y_ambuja.isnull().sum() == 0

y_ambuja.head()

Ótimo trabalho! Agora que temos uma função de tratamento, vamos obter os retornos da Suzlon Energy também.

### Exercício:
Use sua função `wrangle_data` para obter os retornos dos 2.500 dias de negociação mais recentes da Suzlon Energy. Atribua os resultados a `y_suzlon`.

In [ ]:
y_suzlon = ...

print("y_suzlon type:", type(y_suzlon))
print("y_suzlon shape:", y_suzlon.shape)
y_suzlon.head()

## Explorar

Vamos recriar o gráfico da série temporal de volatilidade que fizemos na última aula para que tenhamos um recurso visual para discutir o que é volatilidade.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot returns for `df_suzlon` and `df_ambuja`
y_suzlon.plot(ax=ax, label="SUZLON")
y_ambuja.plot(ax=ax, label="AMBUJACEM")

# Label axes
plt.xlabel("Date")
plt.ylabel("Return")

# Add legend
plt.legend();

O gráfico acima mostra como os retornos mudam ao longo do tempo. Isso pode parecer um conceito totalmente novo, mas se visualizarmos os retornos sem considerar o tempo, as coisas começarão a parecer familiares.

### Exercício:
Crie um histograma de `y_ambuja` com 25 bins. Certifique-se de rotular o eixo x como `"Retornos"`, o eixo y como `"Frequência [contagem]"` e use o título `"Distribuição dos Retornos Diários da Ambuja Cement"`.

In [ ]:
# Create histogram of `y_ambuja`, 25 bins


# Add axis labels



# Add title


Essa é uma forma familiar! Acontece que os retornos seguem uma distribuição quase normal, centrada em `0`. **Volatilidade** é a medida da dispersão desses retornos em torno da média. Em outras palavras, a volatilidade em finanças é a mesma coisa que o desvio padrão em estatísticas.

Vamos começar medindo a volatilidade diária das nossas duas ações. Como nossa frequência de dados também é diária, isso será exatamente o mesmo que calcular o desvio padrão.

### Exercício:
Calcule a volatilidade diária para Suzlon e Ambuja, atribuindo os resultados às variáveis `suzlon_daily_volatility` e `ambuja_daily_volatility`, respectivamente.

In [ ]:
suzlon_daily_volatility = ...
ambuja_daily_volatility = ...

print("Suzlon Daily Volatility:", suzlon_daily_volatility)
print("Ambuja Daily Volatility:", ambuja_daily_volatility)

Parece que Suzlon é mais volátil do que Ambuja. Isso reforça o que vimos em nosso gráfico de série temporal, onde os retornos da Suzlon têm uma dispersão muito maior.

Embora a volatilidade diária seja útil, os investidores também se interessam pela volatilidade em outros períodos — como a volatilidade anual. Lembre-se de que um ano não tem 365 dias para o mercado de ações. Após excluir finais de semana e feriados, a maioria dos mercados tem apenas 252 dias de negociação.

Então, como fazemos a transição da volatilidade diária para a volatilidade anual? Da mesma forma que calculamos o desvio padrão para nosso experimento de vários dias no Projeto 7!

### Exercício:
Calcule a volatilidade anual para Suzlon e Ambuja, atribuindo os resultados a `suzlon_annual_volatility` e `ambuja_annual_volatility`, respectivamente.

In [ ]:
suzlon_annual_volatility = ...
ambuja_annual_volatility = ...

print("Suzlon Annual Volatility:", suzlon_annual_volatility)
print("Ambuja Annual Volatility:", ambuja_annual_volatility)

Novamente, Suzlon apresenta uma volatilidade maior do que Ambuja. O que você acha que significa que a volatilidade anual é maior do que a volatilidade diária?

Como estamos lidando com dados de séries temporais, outra maneira de analisar a volatilidade é calculá-la usando uma janela deslizante. Começaremos a nos concentrar exclusivamente na Ambuja Cement.

### Exercício:
Calcule a volatilidade móvel para `y_ambuja`, usando uma janela de 50 dias. Atribua o resultado a `ambuja_rolling_50d_volatility`.

In [ ]:
ambuja_rolling_50d_volatility = ...

print("rolling_50d_volatility type:", type(ambuja_rolling_50d_volatility))
print("rolling_50d_volatility shape:", ambuja_rolling_50d_volatility.shape)
ambuja_rolling_50d_volatility.head()

Desta vez, vamos nos concentrar na Ambuja Cement.

### Exercício:
Crie um gráfico de série temporal mostrando os retornos diários da Ambuja Cement e a volatilidade móvel de 50 dias. Certifique-se de rotular seus eixos e incluir uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot `y_ambuja`


# Plot `ambuja_rolling_50d_volatility`


# Add x-axis label


# Add legend


Aqui podemos ver que a volatilidade aumenta quando os retornos mudam drasticamente — tanto para cima quanto para baixo. Por exemplo, podemos observar um grande aumento na volatilidade em maio de 2020, quando houve vários dias de grandes retornos negativos. Também podemos ver a volatilidade diminuir em agosto de 2022, quando há apenas pequenas mudanças diárias nos retornos.

Esse gráfico revela um problema. Queremos usar os retornos para ver se uma alta volatilidade em um dia está associada a uma alta volatilidade no dia seguinte. Mas a alta volatilidade é causada por grandes mudanças nos retornos, que podem ser positivas ou negativas. Como podemos avaliar números negativos e positivos juntos sem que eles se cancelarem? Uma solução é tomar o valor absoluto dos números, que é o que fazemos para calcular métricas de desempenho, como o erro absoluto médio. A outra solução, que é mais comum neste contexto, é elevar todos os valores ao quadrado.

### Exercício:
Crie um gráfico de série temporal dos retornos ao quadrado em `y_ambuja`. Não se esqueça de rotular seus eixos.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot squared returns


# Add axis labels



Perfeito! Agora é muito mais fácil ver que (1) temos períodos de alta e baixa volatilidade, e (2) os dias de alta volatilidade tendem a se agrupar. Esta é uma situação perfeita para usar um modelo GARCH.

Um modelo GARCH é um tipo de modelo ARMA que aprendemos na Aula 3.4. Ele tem um parâmetro `p` que lida com correlações em etapas de tempo anteriores e um parâmetro `q` para lidar com eventos de "choque". Ele também utiliza a noção de defasagem. Para ver quantas defasagens devemos ter em nosso modelo, devemos criar um gráfico ACF e PACF — mas usando os retornos ao quadrado.

### Exercício:
Crie um gráfico ACF dos retornos ao quadrado da Ambuja Cement. Certifique-se de rotular seu eixo x como `"Defasagem [dias]"` e seu eixo y como `"Coeficiente de Correlação"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared returns


# Add axis labels



### Exercício:
Crie um gráfico PACF dos retornos ao quadrado da Ambuja Cement. Certifique-se de rotular seu eixo x como `"Defasagem [dias]"` e seu eixo y como `"Coeficiente de Correlação"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create PACF of squared returns


# Add axis labels



Em nosso PACF, parece que uma defasagem de 3 seria um bom ponto de partida.

Normalmente, neste ponto do processo de construção do modelo, dividiríamos nossos dados em conjuntos de treinamento e teste e, em seguida, estabeleceríamos uma linha de base. Não desta vez. Isso ocorre porque a entrada do nosso modelo e sua saída são duas medições diferentes. Usaremos **retornos** para treinar nosso modelo, mas queremos que ele preveja **volatilidade**. Se criássemos um conjunto de teste, não teríamos os "valores verdadeiros" que precisaríamos para avaliar o desempenho do nosso modelo. Portanto, desta vez, iremos direto para a iteração.

## Dividir

A última coisa que precisamos fazer antes de construir nosso modelo é criar um conjunto de treinamento. Observe que não vamos criar um conjunto de teste aqui. Em vez disso, usaremos todo o `y_ambuja` para conduzir a validação em avanço contínuo após termos construído nosso modelo.

### Exercício:
Crie um conjunto de treinamento `y_ambuja_train` que contenha os primeiros 80% das observações em `y_ambuja`.

In [ ]:
cutoff_test = ...
y_ambuja_train = ...

print("y_ambuja_train type:", type(y_ambuja_train))
print("y_ambuja_train shape:", y_ambuja_train.shape)
y_ambuja_train.tail()

# Construir Model

Assim como fizemos da última vez que construímos um modelo como este, começaremos a iterar.

## Iterar

### Exercício:
Construa e ajuste um modelo GARCH usando os dados em `y_ambuja`. Comece com `3` como valor para `p` e `q`. Em seguida, use o resumo do modelo para avaliar seu desempenho e experimente outras defasagens.

In [ ]:
# Build and train model
model = ...
print("model type:", type(model))

# Show model summary


__Dica:__ Você pode acessar os scores AIC e BIC programaticamente. Cada <code>ARCHModelResult</code> possui um atributo <code>.aic</code> e <code>.bic</code>. Experimente você mesmo: insira <code>model.aic</code> ou <code>model.bic</code>.

Agora que decidimos sobre um modelo, vamos visualizar suas previsões, juntamente com os retornos da Ambuja.

### Exercício:
Crie um gráfico de série temporal com os retornos da Ambuja e a volatilidade condicional do seu `modelo`. Certifique-se de incluir rótulos nos eixos e adicionar uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot `y_ambuja_train`


# Plot conditional volatility * 2


# Plot conditional volatility * -2


# Add axis labels



# Add legend


Visualmente, nosso modelo parece bastante bom, mas devemos examinar os resíduos, apenas para garantir. No caso dos modelos GARCH, precisamos olhar para os resíduos padronizados.

### Exercício:
Crie um gráfico de série temporal dos resíduos padronizados do seu `modelo`. Certifique-se de incluir rótulos nos eixos e uma legenda.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot standardized residuals


# Add axis labels



# Add legend


Esses resíduos parecem bons: eles têm uma média e uma dispersão consistentes ao longo do tempo. Vamos verificar sua normalidade usando um histograma.

### Exercício:
Crie um histograma com 25 bins dos resíduos padronizados do seu modelo. Certifique-se de rotular seus eixos e usar um título.

In [ ]:
# Create histogram of standardized residuals, 25 bins


# Add axis labels



# Add title


Nossa última visualização será a ACF dos resíduos padronizados. Assim como fizemos com nossa primeira ACF, precisaremos quadrar os valores aqui também.

### Exercício:
Crie um gráfico ACF do quadrado dos seus resíduos padronizados. Não se esqueça de rotular os eixos!

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Create ACF of squared, standardized residuals


# Add axis labels



Excelente! Parece que este modelo está pronto para uma avaliação final.

## Avaliar

Para avaliar nosso modelo, faremos validação contínua. Antes de fazermos isso, vamos dar uma olhada em como este modelo retorna suas previsões.

### Exercício:
Crie uma previsão de um dia a partir do seu `modelo` e atribua o resultado à variável `one_day_forecast`.

In [ ]:
one_day_forecast = ...

print("one_day_forecast type:", type(one_day_forecast))
one_day_forecast

Existem duas coisas que precisamos ter em mente aqui. Primeiro, nossa previsão do `modelo` mostra a **variância** prevista, não o **desvio padrão** / **volatilidade**. Portanto, precisaremos tirar a raiz quadrada do valor. Em segundo lugar, a previsão está na forma de um DataFrame. Ele possui um DatetimeIndex, e a data é o último dia para o qual temos dados de treinamento. A coluna `"h.1"` representa "horizonte 1", ou seja, a previsão do nosso modelo para o dia seguinte. Precisaremos manter tudo isso em mente ao reformular essa previsão para servir ao usuário final de nossa aplicação.

### Exercício:
Complete o código abaixo para realizar a validação contínua no seu `modelo`. Em seguida, execute o bloco de código a seguir para visualizar as previsões do modelo em teste.

In [ ]:
# Create empty list to hold predictions
predictions = []

# Calculate size of test data (20%)
test_size = int(len(y_ambuja) * 0.2)

# Walk forward
for i in range(test_size):
    # Create test data
    y_train = y_ambuja.iloc[: -(test_size - i)]

    # Train model
    model = ...

    # Generate next prediction (volatility, not variance)
    next_pred = ...

    # Append prediction to list
    predictions.append(next_pred)

# Create Series from predictions list
y_test_wfv = pd.Series(predictions, index=y_ambuja.tail(test_size).index)

print("y_test_wfv type:", type(y_test_wfv))
print("y_test_wfv shape:", y_test_wfv.shape)
y_test_wfv.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

# Plot returns for test data
y_ambuja.tail(test_size).plot(ax=ax, label="Ambuja Return")

# Plot volatility predictions * 2
(2 * y_test_wfv).plot(ax=ax, c="C1", label="2 SD Predicted Volatility")

# Plot volatility predictions * -2
(-2 * y_test_wfv).plot(ax=ax, c="C1")

# Label axes
plt.xlabel("Date")
plt.ylabel("Return")

# Add legend
plt.legend();

Isso parece muito bom. Nossas previsões de volatilidade parecem seguir as mudanças nos retornos ao longo do tempo. Isso é especialmente claro no período de baixa volatilidade no verão de 2022 e no período de alta volatilidade no outono de 2022.

Um passo adicional que poderíamos dar para avaliar como o nosso `modelo` se sai nos dados de teste seria plotar a ACF dos resíduos padronizados apenas para o conjunto de teste. Mas você pode fazer esse passo por conta própria.

# Comunicar Resultados

Normalmente, nesta seção, criamos visualizações para um público humano, mas nosso objetivo para *este* projeto é criar uma API para um público *computacional*. Portanto, vamos nos concentrar em transformar as previsões do nosso modelo em formato JSON, que é o que usaremos para enviar previsões em nossa aplicação.

A primeira coisa que precisamos fazer é criar um `DatetimeIndex` para nossas previsões. Usar rótulos como `"h.1"`, `"h.2"`, etc., não funcionará. Mas há duas coisas que precisamos ter em mente. Primeiro, não podemos incluir datas que sejam fins de semana, pois não há negociações nesses dias. E precisaremos escrever nossas datas usando strings que sigam o padrão [ISO 8601](https://en.wikipedia.org/wiki/ISO_8601).

### Exercício:
Abaixo está uma `previsão`, que contém uma previsão de 5 dias do nosso `modelo`. Usando isso como ponto de partida, crie um `índice_de_previsão`. Isso deve ser uma lista com as seguintes 5 datas escritas no formato ISO 8601.

In [ ]:
# Generate 5-day volatility forecast
prediction = model.forecast(horizon=5, reindex=False).variance ** 0.5
print(prediction)

# Calculate forecast start date
start = ...

# Create date range
prediction_dates = ...

# Create prediction index labels, ISO 8601 format
prediction_index = ...

print("prediction_index type:", type(prediction_index))
print("prediction_index len:", len(prediction_index))
prediction_index[:3]

Agora que sabemos como criar o índice, vamos criar uma função para combinar o índice e as previsões, e então retornar um dicionário onde cada chave é uma data e cada valor é uma volatilidade prevista.

### Exercício:
Crie uma função `clean_prediction`. Ela deve receber um DataFrame de previsões de variância como entrada e retornar um dicionário onde cada chave é uma data no formato ISO 8601 e cada valor é a volatilidade prevista. Use a docstring como guia e as instruções `assert` para testar sua função. Quando estiver satisfeito com o resultado, envie-o para o avaliador.

In [ ]:
def clean_prediction():

    """Reformat model prediction to JSON.

    Parameters
    ----------
    prediction : pd.DataFrame
        Variance from a `ARCHModelForecast`

    Returns
    -------
    dict
        Forecast of volatility. Each key is date in ISO 8601 format.
        Each value is predicted volatility.
    """
    # Calculate forecast start date


    # Create date range


    # Create prediction index labels, ISO 8601 format


    # Extract predictions from DataFrame, get square root


    # Combine `data` and `prediction_index` into Series


    # Return Series as dictionary
    return ...

In [ ]:
prediction = model.forecast(horizon=10, reindex=False).variance
prediction_formatted = clean_prediction(prediction)

# Is `prediction_formatted` a dictionary?
assert isinstance(prediction_formatted, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction_formatted.keys())

# Are values correct data type
assert all(isinstance(v, float) for v in prediction_formatted.values())

prediction_formatted

Ótimo trabalho! Agora temos vários componentes para nossa aplicação: classes para obter dados de uma API, classes para armazená-los em um banco de dados e código para construir nosso modelo e limpar nossas previsões. O próximo passo é criar uma classe para nosso modelo e definir os caminhos para a aplicação — ambos serão feitos na próxima lição.

# Model Deployment

Pronto para o deploy! Nas últimas três lições, construímos todas as peças de que precisamos para nossa aplicação. Temos um módulo para obter e armazenar nossos dados. Temos o código para treinar nosso modelo e limpar suas previsões. Nesta lição, vamos juntar todas essas peças e fazer o deploy do nosso modelo com uma API que outros poderão usar para treinar seus próprios modelos e prever a volatilidade. Vamos começar criando um `model` para todo o código que criamos na última lição. Depois, completaremos nosso módulo `main`, que conterá nossa aplicação FastAPI com dois caminhos: um para o treinamento do modelo e outro para a previsão. Vamos começar!

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob

import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository

# Módulo do Modelo


Criamos muito código na última lição para construir, treinar e fazer previsões com o nosso modelo GARCH(1,1). Queremos que esse código seja reutilizável, então vamos colocá-lo em um módulo próprio.

Vamos começar instanciando um repositório que usaremos para testar nosso módulo conforme o desenvolvemos.

### Exercício:
Crie um `SQLRepository` chamado `repo`. Certifique-se de que ele esteja conectado a uma conexão SQLite.

In [ ]:
connection = ...
repo = ...

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

Agora que temos o `repo` pronto, vamos mudar para o nosso módulo `model` e criar uma classe `GarchModel` para armazenar todo o nosso código da última lição.

### Exercício:
No módulo `model`, crie uma definição para uma classe de modelo `GarchModel`. Por enquanto, ela deve ter apenas um método `__init__`. Use o docstring como guia. Quando terminar, teste sua classe usando as instruções `assert` abaixo.

In [ ]:
from model import GarchModel

# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

### Exercício:
Transforme sua função `wrangle_data` da última lição em um método para a classe `GarchModel`. Quando terminar, use as instruções `assert` abaixo para testar o método, obtendo e organizando dados para a loja de departamento [Shoppers Stop](https://www.shoppersstop.com/).

In [ ]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

### Exercício:
Usando seu código da lição anterior, crie um método `fit` para sua classe `GarchModel`. Quando terminar, use o código abaixo para testá-lo.

In [ ]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

### Exercício:
Usando seu código da lição anterior, crie um método `predict_volatility` para sua classe `GarchModel`. Seu método precisará retornar as previsões como um dicionário, então você precisará adicionar sua função `clean_prediction` como um método auxiliar. Quando terminar, teste seu trabalho usando as instruções assert abaixo.

In [ ]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

prediction

As coisas estão indo bem! Existem dois últimos métodos que precisamos adicionar ao nosso `GarchModel` para que possamos salvar um modelo treinado e, em seguida, carregá-lo quando precisarmos. Quando aprendemos sobre salvar e carregar arquivos no Projeto 5, usamos um manipulador de contexto. Desta vez, vamos simplificar o processo usando a [biblioteca joblib](https://joblib.readthedocs.io/en/latest/). Também começaremos a escrever nossos caminhos de arquivo de forma mais programática usando a [biblioteca os](https://docs.python.org/3/library/os.html).

### Exercício:
Crie um método `dump` para sua classe `GarchModel`. Ele deve salvar o modelo atribuído ao atributo `model` na pasta especificada em sua configuração `settings`. Use a docstring como guia e, em seguida, teste seu trabalho abaixo.

In [ ]:
# Save `model_shop` model, assign filename
filename = model_shop.dump()

# Is `filename` a string?
assert isinstance(filename, str)

# Does filename include ticker symbol?
assert model_shop.ticker in filename

# Does file exist?
assert os.path.exists(filename)

filename

### Exercício:
Crie uma função `load` abaixo que receberá um símbolo de ticker como entrada e retornará um modelo. Quando terminar, use a próxima célula para carregar o modelo da Shoppers Stop que você salvou na tarefa anterior.

In [ ]:
def load():

    """Load latest model from model directory.

    Parameters
    ----------
    ticker : str
        Ticker symbol for which model was trained.

    Returns
    -------
    `ARCHModelResult`
    """
    # Create pattern for glob search
    pattern = ...

    # Try to find path of latest model

    model_path = ...

    # Handle possible `IndexError`


    # Load model
    model = ...

    # Return model
    return ...

In [ ]:
# Assign load output to `model`
model_shop = load(ticker="SHOPERSTOP.BSE")

# Does function return an `ARCHModelResult`
assert isinstance(model_shop, ARCHModelResult)

# Check model parameters
model_shop.summary()

### Exercício:
Transforme sua função `load` em um método para sua classe `GarchModel`. Quando terminar, teste o método usando as instruções `assert` abaixo.

In [ ]:
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Check that new `model_shop_test` doesn't have model attached
assert not hasattr(model_shop, "model")

# Load model
model_shop.load()

# Does `model_shop_test` have model attached?
assert hasattr(model_shop, "model")

model_shop.model.summary()

Nosso módulo `model` está pronto! Agora é hora de passar para o curso "principal" e adicionar a peça final à nossa aplicação.

# Módulo Main

Semelhante às aplicações interativas que fizemos no Projeto 4, nosso primeiro passo aqui será criar um objeto `app`. Desta vez, em vez de ser uma aplicação plotly, será uma aplicação FastAPI.

### Exercício:
No módulo `main`, instancie uma aplicação FastAPI chamada `app`.

Para que nosso `app` funcione, precisamos executá-lo em um servidor. Neste caso, executaremos o servidor em nossa máquina virtual usando a biblioteca [uvicorn](https://www.uvicorn.org/).

###Exercício:
Vá para a linha de comando, navegue até o diretório deste projeto e inicie seu servidor de aplicativo digitando o seguinte comando.

```bash
uvicorn main:app --reload --workers 1 --host localhost --port 8008
```

Lembre-se de como a API do AlphaVantage tinha um caminho `"/query"` que acessamos usando uma requisição HTTP `get`? Vamos construir caminhos semelhantes para nossa aplicação. Vamos começar com um exemplo de MVP para que possamos aprender como os caminhos funcionam no FastAPI.

### Exercício:
Crie um caminho `"/hello"` para sua aplicação que retorne uma saudação quando receber uma requisição `get`.

Temos nosso caminho. Vamos realizar uma requisição `get` para ver se funciona.

### Exercício:
Crie uma requisição `get` para acessar o caminho `"/hello"` em execução em `"http://localhost:8008"`.

In [ ]:
url = ...
response = ...

print("response code:", response.status_code)
response.json()

Excelente! Agora vamos começar a construir as partes divertidas.

## `"/fit"` Path

Nossa primeira rota permitirá que o usuário ajuste um modelo aos dados das ações quando ele fizer uma solicitação `post` ao nosso servidor. Eles terão a opção de usar novos dados da AlphaVantage ou dados mais antigos que já estão em nosso banco de dados. Quando um usuário faz uma solicitação, ele recebe uma resposta informando se a operação foi bem-sucedida ou se ocorreu um erro.

Uma coisa muito importante ao construir uma API é garantir que o usuário passe os parâmetros corretos para o aplicativo. Caso contrário, nosso aplicativo pode falhar! O FastAPI funciona bem com a [biblioteca pydantic](https://pydantic-docs.helpmanual.io/), que verifica se cada solicitação tem os parâmetros e tipos de dados corretos. Isso é feito usando classes de dados especiais que precisamos definir. Nossa rota `"/fit"` aceitará a entrada do usuário e, em seguida, retornará uma resposta, então precisamos de duas classes: uma para entrada e outra para saída.

### Exercício:
Defina as classes de dados FitIn e FitOut. A classe FitIn deve herdar da classe BaseModel do pydantic, e a classe FitOut deve herdar da classe FitIn. Certifique-se de incluir dicas de tipo.

Com nossas classes de dados definidas, vamos ver como o pydantic garante que os usuários estão fornecendo a entrada correta e nossa aplicação está retornando a saída correta.

### Exercício:
Use o código abaixo para experimentar suas classes FitIn e FitOut. Sob quais circunstâncias a instanciação delas gera erros? De qual classe ou classes elas são instâncias?

In [ ]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = ...
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = ...
print(fo)

Uma característica interessante do FastAPI é que ele pode funcionar em cenários assíncronos. Isso não é algo que precisamos aprender para este projeto, mas significa que precisamos instanciar um objeto `GarchModel` cada vez que um usuário faz uma solicitação. Para facilitar a codificação, vamos criar uma função para lidar com esse processo para nós.

### Exercício:
Crie uma função `build_model` em seu módulo `main`. Use a docstring como guia e teste sua função abaixo.

In [ ]:
from main import build_model

# Instantiate `GarchModel` with function
model_shop = build_model(ticker="SHOPERSTOP.BSE", use_new_data=False)

# Is `SQLRepository` attached to `model_shop`?
assert isinstance(model_shop.repo, SQLRepository)

# Is SQLite database attached to `SQLRepository`
assert isinstance(model_shop.repo.connection, sqlite3.Connection)

# Is `ticker` attribute correct?
assert model_shop.ticker == "SHOPERSTOP.BSE"

# Is `use_new_data` attribute correct?
assert not model_shop.use_new_data

model_shop

Temos classes de dados, temos uma função `build_model` e tudo o que resta é construir o caminho `"/fit"`. Usaremos nosso caminho `"/hello"` como um template, mas precisaremos incluir mais recursos, como tratamento de erros.

### Exercício:
Crie um caminho `"/fit"` para seu `app`. Ele receberá um objeto `FitIn` como entrada e, em seguida, construirá um `GarchModel` usando a função `build_model`. O modelo irá processar os dados necessários, ajustar-se aos dados e salvar o modelo completo. Por fim, enviará uma resposta na forma de um objeto `FitOut`. Certifique-se de tratar qualquer erro que possa surgir.

Última etapa! Vamos fazer uma solicitação `post` e ver como nosso aplicativo responde.

### Exercício:
Crie uma solicitação `post` para acessar o caminho `"/fit"` em execução em `"http://localhost:8008"`. Você deve treinar um modelo GARCH(1,1) em 2000 observações dos dados da Shoppers Stop que você já baixou. Passe seus parâmetros como um dicionário usando o argumento `json`.

In [ ]:
# URL of `/fit` path
url = ...

# Data to send to path
json = ...
# Response of post request
response = ...
# Inspect response
print("response code:", response.status_code)
response.json()

Boom! Agora podemos treinar modelos usando a API que criamos. A seguir: um caminho para fazer previsões.

## `"/predict"` Path

Para nosso caminho `"/predict"`, os usuários poderão fazer uma solicitação `post` com o símbolo da ação para a qual desejam uma previsão e o número de dias que desejam prever no futuro. Nosso aplicativo retornará uma previsão ou, se houver um erro, uma mensagem explicando o problema.

A configuração será muito semelhante ao nosso caminho `"/fit"`. Começaremos com classes de dados para entrada e saída.

### Exercício:
Crie definições para uma classe de dados `PredictIn` e `PredictOut`. A classe `PredictIn` deve herdar da `BaseModel` do pydantic, e a classe `PredictOut` deve herdar da classe `PredictIn`. Certifique-se de incluir dicas de tipo. Em seguida, use o código abaixo para testar suas classes.

In [ ]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="SHOPERSTOP.BSE", n_days=5)
print(pi)

po = PredictOut(
    ticker="SHOPERSTOP.BSE", n_days=5, success=True, forecast={}, message="success"
)
print(po)

A seguir, vamos criar o caminho. A boa notícia é que poderemos reutilizar nossa função `build_model`.

### Exercício:
Crie um caminho `"/predict"` para seu `app`. Ele deve receber um objeto `PredictIn` como entrada, construir um `GarchModel`, carregar o modelo treinado mais recente para o ticker fornecido e gerar um dicionário de previsões. Em seguida, deve retornar um objeto `PredictOut` com as previsões incluídas. Certifique-se de tratar quaisquer erros que possam surgir.

Último passo, vamos ver o que acontece quando fazemos uma requisição `post`...

### Exercício:
Crie uma requisição `post` para acessar o caminho `"/predict"` rodando em `"http://localhost:8008"`. Você deve obter a previsão de volatilidade para 5 dias da Shoppers Stop. Quando estiver satisfeito, envie seu trabalho para o avaliador.

In [ ]:
# URL of `/predict` path
url = ...
# Data to send to path
json = ...
# Response of post request
response = ...
# Response JSON to be submitted to grader
submission = response.json()
# Inspect JSON
submission

Conseguimos! Melhor dizendo, **você** conseguiu. Você obteve dados da API AlphaVantage, armazenou-os em um banco de dados SQL, construiu e treinou um modelo GARCH para prever volatilidade e criou sua própria API para fornecer previsões do seu modelo. Isso é engenharia de dados, ciência de dados e implantação de modelos, tudo em um único projeto. Se você ainda não fez isso, agora é um bom momento para se dar um tapinha nas costas. Você definitivamente merece.